In [1]:
#hide
#default_exp vis.curtailment

In [2]:
import pandas as pd

from ElexonDataPortal.api import Client

In [3]:
client = Client()
df_detsysprices = client.get_DETSYSPRICES()

df_detsysprices.head()

DETSYSPRICES: 100%|██████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.68it/s]


,local_datetime,recordType,settlementDate,settlementPeriod,cost,volume,adjuster,value,type,priceDerivationCode,...,nivAdjustedVolume,parAdjustedvolume,finalPrice,transmissionLossMultiplier,tlmAdjustedVolume,tlmAdjustedCost,totalOfTlmAdjustedVolume,totalOfTlmAdjustedCost,bidPrice,bidVolume
0,2020-01-01 00:00:00+00:00,MAIN PRICE SUMMARY,2020-01-01,1,51.33,1.008,0.00,50.90000,50.90000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-01 00:00:00+00:00,MARKET PRICE SUMMARY,2020-01-01,1,27680.69,784.600,NaN,35.28000,50.90000,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01 00:00:00+00:00,OFFER,2020-01-01,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20.849,0.384,50.90000,1.0084058,0.387,19.69,1.008,51.33,NaN,NaN
3,2020-01-01 00:00:00+00:00,OFFER,2020-01-01,1,NaN,NaN,NaN,NaN,NaN,NaN,...,33.488,0.616,50.90000,1.0084058,0.621,31.63,1.008,51.33,NaN,NaN
4,2020-01-01 00:00:00+00:00,OFFER,2020-01-01,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,0.000,0.00000,0.0000000,0.000,0.00,1.008,51.33,NaN,NaN


In [4]:
df_detsysprices.iloc[3]

local_datetime                2020-01-01 00:00:00+00:00
recordType                                        OFFER
settlementDate                               2020-01-01
settlementPeriod                                      1
cost                                                NaN
volume                                              NaN
adjuster                                            NaN
value                                               NaN
type                                                NaN
priceDerivationCode                                 NaN
index                                                 2
id                                             T_EECL-1
acceptanceId                                     115310
bidOfferPairId                                        1
cadlFlag                                              F
soFlag                                                F
storFlag                                              F
repricedIndicator                               

In [5]:
#exports
flatten_list = lambda list_: [item for sublist in list_ for item in sublist]

def get_wf_ids(dictionary_url='https://raw.githubusercontent.com/OSUKED/Power-Station-Dictionary/main/data/output/power_stations.csv'):
    df_dictionary = pd.read_csv(dictionary_url)
    wf_ids = flatten_list(df_dictionary.query('fuel_type=="wind"')['sett_bmu_id'].str.split(', ').to_list())
    
    return wf_ids

In [6]:
wf_ids = get_wf_ids()

wf_ids[:5]

['T_ACHRW-1', 'T_AKGLW-2', 'E_AIRSW-1', 'E_ASHWW-1', 'T_ANSUW-1']

In [7]:
#exports
def get_curtailed_wfs_df(
    api_key: str=None,
    start_date: str = '2020-01-01',
    end_date: str = '2020-01-01 1:30',
    wf_ids: list=None,
    dictionary_url: str='https://raw.githubusercontent.com/OSUKED/Power-Station-Dictionary/main/data/output/power_stations.csv'
):
    if wf_ids is None:
        wf_ids = get_wf_ids(dictionary_url=dictionary_url)
        
    if isinstance(start_date, str):
        start_date = pd.to_datetime(start_date).tz_localize('Europe/London')
        
    if isinstance(end_date, str):
        end_date = pd.to_datetime(end_date).tz_localize('Europe/London')
        
    client = Client()
    df_detsysprices = client.get_DETSYSPRICES(start_date, end_date)

    df_curtailed_wfs = (df_detsysprices
                        .query('recordType=="BID" & soFlag=="T" & id in @wf_ids')
                        .astype({'bidVolume': float})
                        .groupby(['local_datetime', 'id'])
                        ['bidVolume']
                        .sum()
                        .reset_index()
                        .pivot('local_datetime', 'id', 'bidVolume')
                       )
    
    df_curtailed_wfs = df_curtailed_wfs.reindex(pd.date_range(min(start_date, df_curtailed_wfs.index.min()), 
                                                              max(end_date-pd.Timedelta(minutes=30), df_curtailed_wfs.index.max()),
                                                              tz='Europe/London'))
    
    df_curtailed_wfs.index.name = 'local_datetime'
    
    return df_curtailed_wfs

In [8]:
df_curtailed_wfs = get_curtailed_wfs_df()

df_curtailed_wfs

DETSYSPRICES: 100%|██████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.05it/s]


id,T_EDINW-1
local_datetime,
2020-01-01 00:00:00+00:00,-5.8


In [9]:
curtailed_wfs_fp = '../data/curtailed_wfs.csv'
overwrite_curtailed_wfs_csv = False

if overwrite_curtailed_wfs_csv == True:
    df_curtailed_wfs.to_csv(curtailed_wfs_fp)

In [10]:
#exports
def load_curtailed_wfs(
    curtailed_wfs_fp: str='data/curtailed_wfs.csv'
):
    df_curtailed_wfs = pd.read_csv(curtailed_wfs_fp)

    df_curtailed_wfs = df_curtailed_wfs.set_index('local_datetime')
    df_curtailed_wfs.index = pd.to_datetime(df_curtailed_wfs.index, utc=True).tz_convert('Europe/London')
    
    return df_curtailed_wfs

In [11]:
df_curtailed_wfs = load_curtailed_wfs(curtailed_wfs_fp)

df_curtailed_wfs.head()

,T_EDINW-1,E_BABAW-1,E_BETHW-1,E_BRYBW-1,E_BTUIW-2,E_CLFLW-1,E_HRHLW-1,E_MOYEW-1,T_AKGLW-2,T_ARCHW-1,...,E_BRDUW-1,T_RREW-1,T_BRBEO-1,T_GANW-11,T_GANW-13,T_GANW-22,T_GANW-24,T_GRGBW-1,T_GRGBW-2,T_GRGBW-3
local_datetime,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00+00:00,-5.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02 00:00:00+00:00,-8.75,NaN,NaN,NaN,NaN,NaN,-8.992,NaN,NaN,-28.467,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03 00:00:00+00:00,-0.50,NaN,-9.017,NaN,NaN,NaN,-8.483,NaN,NaN,-15.083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-04 00:00:00+00:00,-3.00,NaN,-8.992,NaN,-20.008,-3.508,-13.000,NaN,NaN,-48.517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-05 00:00:00+00:00,-9.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
most_recent_ts = df_curtailed_wfs.index.max()

start_date = most_recent_ts + pd.Timedelta(minutes=30)
end_date = start_date + pd.Timedelta(days=7)

client = Client()
df_curtailed_wfs_wk = get_curtailed_wfs_df(start_date=start_date, end_date=end_date, wf_ids=wf_ids)

df_curtailed_wfs_wk.head()

DETSYSPRICES: 100%|██████████████████████████████████████████████████████████████████| 336/336 [07:32<00:00,  1.35s/it]


id,E_BETHW-1,E_BRDUW-1,E_BTUIW-2,E_CLDRW-1,E_HRHLW-1,T_AKGLW-2,T_ARCHW-1,T_BHLAW-1,T_BLLA-1,T_BLLA-2,...,T_HRSTW-1,T_KLGLW-1,T_LCLTW-1,T_MKHLW-1,T_SANQW-1,T_STLGW-1,T_STLGW-2,T_STRNW-1,T_WHILW-1,T_WHILW-2
local_datetime,,,,,,,,,,,,,,,,,,,,,
2020-12-23 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-24 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-25 22:00:00+00:00,NaN,NaN,NaN,NaN,-11.017,NaN,NaN,NaN,-36.025,NaN,...,NaN,-107.042,NaN,NaN,NaN,NaN,NaN,-15.5,-18.225,NaN
2020-12-26 22:00:00+00:00,-5.508,-27.5,-17.5,-12.5,-13.016,-31.5,-46.533,-41.484,-48.625,-27.975,...,-62.5,-114.000,NaN,-17.0,NaN,NaN,NaN,NaN,NaN,-72.658
2020-12-27 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#exports
def add_next_week_of_data_to_curtailed_wfs(
    curtailed_wfs_fp: str='data/curtailed_wfs.csv',
    save_data: bool=True,
):
    df_curtailed_wfs = load_curtailed_wfs(curtailed_wfs_fp)
    most_recent_ts = df_curtailed_wfs.index.max()

    start_date = most_recent_ts + pd.Timedelta(minutes=30)
    end_date = start_date + pd.Timedelta(days=7)

    client = Client()
    df_curtailed_wfs_wk = get_curtailed_wfs_df(start_date=start_date, end_date=end_date, wf_ids=wf_ids)

    df_curtailed_wfs = df_curtailed_wfs.append(df_curtailed_wfs_wk)    
    df_curtailed_wfs.to_csv(curtailed_wfs_fp)
    
    return df_curtailed_wfs

In [14]:
df_curtailed_wfs = add_next_week_of_data_to_curtailed_wfs('../data/curtailed_wfs.csv')

DETSYSPRICES: 100%|██████████████████████████████████████████████████████████████████| 336/336 [06:31<00:00,  1.17s/it]


In [15]:
reached_latest_wk = False
update_freq = 5
update_counter = 1
df_curtailed_wfs = load_curtailed_wfs(curtailed_wfs_fp)

while reached_latest_wk == False:
    df_curtailed_wfs = add_next_week_of_data_to_curtailed_wfs('../data/curtailed_wfs.csv')

    most_recent_ts = df_curtailed_wfs.index.max()
    reached_latest_wk = (pd.Timestamp.now(tz='Europe/London') - most_recent_ts).total_seconds() <= 60*60*24*7

    if update_counter == update_freq:
        update_counter = 1
        print(f"Have now reached: {most_recent_ts.strftime('%Y-%m-%d %H:%M')}")
    else:
        update_counter += 1

DETSYSPRICES:  90%|███████████████████████████████████████████████████████████▏      | 301/336 [06:07<00:42,  1.22s/it]


ConnectionError: HTTPSConnectionPool(host='api.bmreports.com', port=443): Max retries exceeded with url: /BMRS/DETSYSPRICES/v1?APIKey=pqth1yrw1rkh5eb&SettlementDate=2021-01-05&SettlementPeriod=11&ServiceType=xml (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001E2D99D68E0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
# want to plot a map of total curtailment in the last year
# want to plot an aggregated time-series of the last year
# if there has been curtailment in the last week then should show it in full
# want to report total curtailment to date on an annual basis

In [ ]:
df_curtailed_wfs = load_curtailed_wfs(curtailed_wfs_fp)
(-df_curtailed_wfs.sum(axis=1)).plot()

In [ ]:
(-df_curtailed_wfs.sum(axis=1)).plot()

In [ ]:
# def update_curtailed_wfs_data():
start_date = df_curtailed_wfs.index.max()
end_date = (pd.Timestamp.now(tz='Europe/London') + pd.Timedelta(minutes=60)).round('60min')

client = Client()
df_detsysprices = client.get_DETSYSPRICES(start_date=start_date, end_date=end_date)

df_detsysprices.head()